## Conectar a la base de datos

In [13]:
from cassandra.cluster import Cluster
from cassandra import ConsistencyLevel
from cassandra.query import SimpleStatement

# Conexión al clúster de Cassandra con los nombres de host de tus nodos
cluster = Cluster(['127.0.0.1'], port=9042)

# Crear una sesión
session = cluster.connect()

# Seleccionar el keyspace (si ya tienes uno creado)
session.set_keyspace('videojuego')

## User finish dungeon

In [2]:
# Entrada
dungeon_id: int = 0
date: str = '2020-04-18 04:46:20'
time_minutes: float = 0.1
email: str = 'aaoki@example.com'

In [ ]:
def finish_dungeon(session, dungeon_id, date, time_minutes, email):
    # Establecer consistencia más fuerte
    # session.default_consistency_level = ConsistencyLevel.ALL

    # Hall of Fame
    rows = session.execute(SimpleStatement('SELECT Email, Tiempo, Nombre_mazmorra, Pais FROM Hall_of_fame WHERE Mazmorra_id=%s ALLOW FILTERING;',consistency_level=ConsistencyLevel.ALL), (dungeon_id, ))
    Nombre_mazmorra = rows.one().nombre_mazmorra
    min_time = time_minutes
    min_row = None

    for row in rows:  # Buscar el peor tiempo en el Top 5
        if row[1] > min_time:
            min_row = row
            min_time = row.tiempo
    
    if min_row is not None:  # Si entra en el Top 5
        # Eliminar el tiempo más alto (el peor tiempo en el Top 5)
        delete_query = SimpleStatement('DELETE FROM Hall_of_fame WHERE Mazmorra_id=%s AND Email=%s AND Pais=%s',consistency_level=ConsistencyLevel.ALL)
        session.execute(delete_query, (dungeon_id, min_row.email, min_row.pais))

        # Insertar el nuevo tiempo
        datos_user = session.execute(SimpleStatement('SELECT Pais, Nombre_usuario FROM Usuarios WHERE Email=%s',consistency_level=ConsistencyLevel.ALL), (email, ))
        pais, nombre = datos_user.one()

        insert_query = SimpleStatement('INSERT INTO Hall_of_fame (Pais, Mazmorra_id, Email, Nombre_usuario, Tiempo, Fecha, Nombre_mazmorra) VALUES (%s, %s, %s, %s, %s, %s, %s)',consistency_level=ConsistencyLevel.ALL)
        session.execute(insert_query, (pais, dungeon_id, email, nombre, time_minutes, date, Nombre_mazmorra))
        print('Quitamos último, metemos al nuevo')
    else:
        print('No está en top 5')


    # Estadísticas
    insert_query = SimpleStatement('INSERT INTO Statistic (Email, Mazmorra_id, Tiempo, Fecha) VALUES (%s, %s, %s, %s)', consistency_level=ConsistencyLevel.ALL)
    session.execute(insert_query, (email, dungeon_id, time_minutes, date))
    print('Estadísticas actualizadas')


In [4]:
finish_dungeon(session, dungeon_id, date, time_minutes, email)

/tmp/ipykernel_406349/3377718518.py:3: DeprecationWarning: Setting the consistency level at the session level will be removed in 4.0. Consider using execution profiles and setting the desired consistency level to the EXEC_PROFILE_DEFAULT profile.
  session.default_consistency_level = ConsistencyLevel.ALL


Quitamos último, metemos al nuevo
Estadísticas actualizadas


## User kills monster during Horde event

In [10]:
# Entrada
event_id: int =  2
email: str = 'jyamashita@example.org'
monster_id = 2

In [ ]:
def kill_monster(session, event_id, email, monster_id):
    # Establecer consistencia baja para maximizar la velocidad
    # session.default_consistency_level = ConsistencyLevel.ONE
    # Obtener la información del usuario desde la tabla Usuarios
    datos_user = session.execute(SimpleStatement('SELECT Pais, Nombre_usuario FROM Usuarios WHERE Email=%s', consistency_level=ConsistencyLevel.ONE), (email,))
    if not datos_user:
        print("No se encontró al usuario con el email proporcionado.")
        return
    
    pais, nombre = datos_user.one()  # Recuperar el país y nombre del usuario

    # Incrementar el contador de kills en la tabla Top_horde
    # Si el registro ya existe, usamos UPDATE, si no, INSERTAMOS un nuevo registro.
    query_check = SimpleStatement('SELECT N_killed FROM Top_horde WHERE Evento_id=%s AND Pais=%s AND Email=%s ALLOW FILTERING;', consistency_level=ConsistencyLevel.ONE)
    rows = session.execute(query_check, (event_id, pais, email))

    # session.default_consistency_level = ConsistencyLevel.ANY
    if rows:  # Si ya existe el registro, incrementamos N_killed
        current_kills = rows.one().n_killed
        # Eliminar la fila existente
        query_delete = SimpleStatement('DELETE FROM Top_horde WHERE Evento_id=%s AND Pais=%s AND N_killed= %s AND Email=%s', consistency_level=ConsistencyLevel.ANY)
        session.execute(query_delete, (event_id, pais, current_kills, email))
        # Insertar una nueva fila con el valor actualizado
        query_insert = SimpleStatement('INSERT INTO Top_horde (Evento_id, Pais, Email, N_killed, Nombre_usuario) VALUES (%s, %s, %s, %s)', consistency_level=ConsistencyLevel.ANY)
        session.execute(query_insert, (event_id, pais, email, current_kills + 1, nombre))

        print("Incrementamos el contador de kills en Top_horde.")
    else:  # Si no existe, lo creamos con un INSERT
        query_insert = SimpleStatement('INSERT INTO Top_horde (Evento_id, Pais, N_killed, Email, Nombre_usuario) VALUES (%s, %s, %s, %s, %s)', consistency_level=ConsistencyLevel.ANY)
        session.execute(query_insert, (event_id, pais, 1, email, nombre))
        print("Añadimos un nuevo registro en Top_horde con el primer kill.")

    # Mensaje de confirmación
    print('Añadimos la kill para el usuario:', email)

In [14]:
kill_monster(session, event_id, email, monster_id)

InvalidRequest: Error from server: code=2200 [Invalid query] message="PRIMARY KEY column "email" cannot be restricted as preceding column "n_killed" is not restricted"

## Cerramos conexión

In [9]:
# Cerrar la conexión
cluster.shutdown()